# Setup

In [37]:
import numpy as np
import pandas as pd
import pickle
import tqdm.auto as tqdm
from pprint import pprint

In [25]:
letter_grades = ["A+", "A", "A-", "B+", "B", "B-", "C+", "C", "C-", "D+", "D",
                  "D-", "NP", "W"]
WCC_PD_PATH = "../data/wcc_pd.pkl"

# Dataset of the last 5 years record

In [17]:
with open(WCC_PD_PATH, "rb") as file:
    raw_data = pickle.load(file)
    
raw_data.columns

Index(['acad_career', 'strm', 'stdnt_enrl_status', 'unt_taken', 'unt_billing',
       'crse_grade_input', 'earn_credit', 'emplid', 'subject', 'catalog_nbr',
       'crse_acad_org', 'sex'],
      dtype='object')

Winter is 4, Spring is 6, Fall is 2, Summer is 8?
e.g. 1198 is 18-19, summer quarter. 1196 is 18-19, spring quarter

In [18]:
def get_course_name (row):
    if row['subject'] != None and row['catalog_nbr'] != None:
        return row['subject']+row['catalog_nbr']
    return ""

def dropped_function (row):
    if row['stdnt_enrl_status'] == 'D':
        return 1
    return 0

In [26]:
# we encode these grades as y = 1, otherwise y = 0
y1_grades = ["A+", "A", "A-", "B+"]

def getStudentFeatures(subject, number, select_majors=False):
    course_entries = raw_data.loc[(raw_data['subject'] == subject) & (raw_data['catalog_nbr'] == number) & # all course entries that are about the certain course we want
                                  (raw_data['strm'] >= 1162) & # taken in 15-16 fall or later
                                  ((raw_data['crse_grade_input'].isin(letter_grades)) | (raw_data['strm'] == 1204)) & # Grade needs to be a letter or it's the latest quarter (nan)
                                  (raw_data['strm'] % 10 != 8) & (raw_data['stdnt_enrl_status'] != 'D')] # not dropped and not summer
#     return course_entries
    students = course_entries.emplid # Students from the course entries
    students_prev_classes = raw_data.loc[raw_data['emplid'].isin(students)] # Other classes taken by these students
    if select_majors:
        students_prev_classes = students_prev_classes.loc[students_prev_classes['subject'].isin(['CME', 'CS', 'EE', 'MATH', 'STATS'])]
    possible_prev_classes = students_prev_classes.apply(lambda row: get_course_name(row), axis=1) # List of courses taken by these students
    possible_prev_classes = possible_prev_classes.unique().tolist() # Focus on unique courses
    possible_prev_classes.append("strm") # For now add term to the data of when the course was taken
    possible_prev_classes.append("female") # Add gender
    possible_prev_classes.append("emplid") # Add emplid
    possible_prev_classes.append("y") # Add the target to the data. Will split it later
    a = np.zeros(shape=(len(course_entries),len(possible_prev_classes))) # all 0 initialized features matrix. +2 for female and strm
    dataset = pd.DataFrame(a,columns=possible_prev_classes) # same as above, but in pandas to allow indexing by course name
#     print(len(course_entries))
#     print('looping')
    for counter, idx in enumerate(course_entries.index): # loop over all class entries
#         print("student: ", counter, " of ", len(course_entries))
        grade = course_entries.loc[[idx]]['crse_grade_input']
        gradeStr = grade.tolist()[0]
        if (gradeStr in y1_grades):
            dataset['y'][counter] = 1
        # fill predictor matrix X
        std_id = course_entries.loc[[idx]]['emplid'] # Get the student id
        quarter = course_entries.loc[[idx]]['strm'] # Get the term
        female = course_entries.loc[[idx]]['sex'] # Get the student gender
        student_courses = students_prev_classes.loc[students_prev_classes['emplid'] == std_id.tolist()[0]] # Get all courses the student took
        student_courses = student_courses.loc[student_courses['strm'] < quarter.tolist()[0]] # Filter the courses to be only before the course of interest
        if select_majors:
            student_courses = student_courses.loc[student_courses['subject'].isin(['CME', 'CS', 'EE', 'MATH', 'STATS'])]
        if (female.tolist()[0] == 'M'):
            dataset["female"][counter] = 0
        else:
            dataset["female"][counter] = 1
        dataset["strm"][counter] = quarter.tolist()[0]
        dataset["emplid"][counter] = std_id.tolist()[0]
        for j in student_courses.index:# Loop over the student's courses
            course_name = student_courses.loc[[j]]['subject']+student_courses.loc[[j]]['catalog_nbr'] # Get the course name
            grade = student_courses.loc[[j]]['crse_grade_input']
            gradeStr = grade.tolist()[0]
            if (gradeStr in y1_grades):
                dataset.iloc[counter, dataset.columns.get_loc(course_name.tolist()[0])] = 1 # Set the course name in X to the new value (or if it was already passed keep the value)
#             dev_break = True
    return dataset

In [27]:
#dataset includes a y column, which is the target. It also includes strm and emplid columns that should
#be dropped before running any prediction
dataset_110 = getStudentFeatures('CS', '110')
dataset_110

/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anacon

,MATH51,CHEM31X,CS106X,CHEM33,PHIL1,CS103,CHEM35,CS109,CS161,CS107,...,BIO153,MS&E212,PHYSICS212,PHYSICS331,LINGUIST205A,MATH226,strm,female,emplid,y
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,0.0,$2a$15$.iQPCHeeuyLD3TIqJRk4j.LU0IjGYumSdFkAEUf...,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1192.0,0.0,$2a$15$5tXEOl2owViV9E5K1pJ/Luf.44w/Ci69ZBbFT6o...,1.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1204.0,0.0,$2a$15$9kztkwAFJfAzPEU6vZlrpe4tgktaXQtimAbnzd2...,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1186.0,0.0,$2a$15$aHTw2jbPTbRKGeDBfcQ25.KPs1kJF/UqcPukXAC...,0.0
4,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1194.0,1.0,$2a$15$CgRoc5cbNZ0QTWMIqNSbu..3w/a0GkYDa3ktG1x...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1204.0,0.0,zgAICyM6fvhWn/C9w/+6Um1cqv8Urgm4nteFXExg9JKxuT...,0.0
1679,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1204.0,1.0,zLKLH/Exzc/w7QreM3/ZE/0o80mxPDWPmrIjZordUAEGCW...,0.0
1680,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1202.0,0.0,zmNzWWYP45nMMEl7vSBuaNnqWtUycLozveGSKlGoXanF8B...,1.0
1681,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1204.0,0.0,zNIibMBEC7HjMsdJhBREHNDAC/XJ5mpAZ0CHO5HpySr00g...,0.0


In [10]:
import collections
collections.Counter(dataset_110.y)

Counter({1.0: 669, 0.0: 1014})

# Try sklearn logistic regression

In [28]:
from sklearn.linear_model import LogisticRegression

In [29]:
# Use the X predictor matrix and y outcome vector from 
# getStudentFeatures, fit a logistic regression model and return
# the list of coefficients, default sorted by absolute value

# Params: sort = ["pos", "neg", "abs"] to sort by largest positive, negative, or
# absolute value of coefficients
def get_coefs(X_train, X_test, y_train, y_test, sort = "abs"):
    model = LogisticRegression(solver="lbfgs")
    model.fit(X_train, y_train)
    train_score = model.score(X_train,y_train) 
    test_score = model.score(X_test,y_test)
    print("training accuracy: %s" % train_score)
    print("test accuracy: %s" % test_score)
    coefs = [(X_test.columns[i], model.coef_[0][i]) for i in range(len(X_test.columns))]
    if sort:
        if sort == "pos":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = True)
        if sort == "neg":
            coefs = sorted(coefs, key = lambda x: x[1], reverse = False)
        if sort == "abs":
            coefs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
    return coefs

In [42]:
def get_and_split_data(subject, course_nbr, test_quarters):
    dataset = getStudentFeatures(subject, course_nbr)
    dataset_grades = dataset.loc[dataset['strm'] < 1204.0] #For now, dropping this quarter
    dataset_train = dataset_grades.loc[~dataset_grades['strm'].isin(test_quarters)]
    dataset_test = dataset_grades.loc[dataset_grades['strm'].isin(test_quarters)]

    dataset_train = dataset_train.drop(["strm", "emplid"], axis=1)
    dataset_test = dataset_test.drop(["strm", "emplid"], axis=1)

    X_train = dataset_train.drop(["y"], axis=1)
    y_train = dataset_train["y"]
    X_test = dataset_test.drop(["y"], axis=1)
    y_test = dataset_test["y"]
    return X_train, X_test, y_train, y_test

In [33]:
X_train, X_test, y_train, y_test = get_and_split_data('CS', '110', [1194.0, 1196.0])
CS110coefs = get_coefs(X_train, X_test, y_train, y_test, 'abs')
CS110coefs[:100]

/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anacon

training accuracy: 0.7738095238095238
test accuracy: 0.6953405017921147


[('PSYCH90', 1.2836920644855365),
 ('CS107', 1.229061155866335),
 ('PHYSICS65', 1.1156710154962242),
 ('OSPKYOTO40M', 1.0583039199887982),
 ('CS246', 1.0183052627893696),
 ('STATS60', -0.9871007624424629),
 ('MS&E221', 0.9841059648650199),
 ('MATH42', 0.979137439196325),
 ('CS376', 0.9424553120430867),
 ('STATS200', 0.9346782156651887),
 ('STATS202', -0.9057196398172502),
 ('ME70', 0.8885504921821742),
 ('PHIL102', -0.8717532269800591),
 ('CS210A', -0.8707752310594443),
 ('CS107E', 0.8395453429648297),
 ('PHYSICS63', -0.8325864568921197),
 ('CS371', 0.8298822280147505),
 ('MATH171', 0.8179725070747609),
 ('PUBLPOL104', 0.8100916862149734),
 ('PHIL1', -0.7808363555654543),
 ('PHIL170', -0.7772683627042153),
 ('PHYSICS25', 0.7663336222798453),
 ('ENGR50', -0.7576951778458186),
 ('EE180', 0.73680506885772),
 ('PHYSICS105', -0.7363919449044659),
 ('CS168', -0.7288668019153931),
 ('CS166', 0.7254345325567374),
 ('ENGR30', -0.7135237844528955),
 ('CHEM35', 0.7040819623668383),
 ('PHIL150', 0

## LASSO logistic regression to reduce nonzero coefficients

In [43]:
def one_course_lasso(subject, course_nbr, test_quarters = [1194.0, 1196.0]):
    X_train, X_test, y_train, y_test = get_and_split_data(subject, course_nbr, test_quarters)
    for i in [10, 3, 1, 0.3, 0.1, 0.03, 0.01]:
      model = LogisticRegression(penalty="l1", solver="saga", C=i, max_iter=1000) 
      model.fit(X_train, y_train)
      train_score = model.score(X_train, y_train)
      test_score = model.score(X_test, y_test)
      coefs = [(X_train.columns[i], model.coef_[0][i]) for i in range(len(X_train.columns))]
      coefs_pos = sorted(coefs, key = lambda x: x[1], reverse = True)
      coefs_neg = sorted(coefs, key = lambda x: x[1], reverse = False)
      coefs_abs = sorted(coefs, key = lambda x: abs(x[1]), reverse = True)
      coefs_nonzero = sum([x[1] != 0 for x in coefs])
      coefs_zero = sum(x[1] == 0 for x in coefs)
      print("\n***** C = %f *****" % i)
      print("training accuracy: %s" % train_score)
      print("test accuracy: %s" % test_score)
      print("Number of nonzero/zero coefficients: %d/%d" % (coefs_nonzero, coefs_zero))
      print("Largest absolute coefficients:")
      pprint(coefs_abs[:3])
      print("Largest positive coefficients:")
      pprint(coefs_pos[:3])
      print("Largest negative coefficients:")
      pprint(coefs_neg[:3])

In [44]:
one_course_lasso("CS", "221")

/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anacon


***** C = 10.000000 *****
training accuracy: 0.8139717425431711
test accuracy: 0.7674418604651163
Number of nonzero/zero coefficients: 254/398
Largest absolute coefficients:
[('PSYCH146', -7.108069344816983),
 ('CS211', -5.834598118383111),
 ('EE169', -5.763255810365245)]
Largest positive coefficients:
[('CS131', 5.085073901270655),
 ('CS210B', 5.012849529679125),
 ('BIO42', 4.8356623824287155)]
Largest negative coefficients:
[('PSYCH146', -7.108069344816983),
 ('CS211', -5.834598118383111),
 ('EE169', -5.763255810365245)]

***** C = 3.000000 *****
training accuracy: 0.7990580847723705
test accuracy: 0.7674418604651163
Number of nonzero/zero coefficients: 191/461
Largest absolute coefficients:
[('CS211', -4.132209580993301),
 ('EE169', -3.206541555031812),
 ('CS131', 3.039215657470817)]
Largest positive coefficients:
[('CS131', 3.039215657470817),
 ('PHIL187', 2.9812303464047174),
 ('CS144', 2.3527309952506763)]
Largest negative coefficients:
[('CS211', -4.132209580993301),
 ('EE169',

In [45]:
one_course_lasso("CS", "229")

/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anaconda3/envs/carta-prereqs/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/coiol/anacon


***** C = 10.000000 *****
training accuracy: 0.8559077809798271
test accuracy: 0.7972972972972973
Number of nonzero/zero coefficients: 222/416
Largest absolute coefficients:
[('PSYCH120', 8.171866137308884),
 ('PSYCH90', -6.817632948191141),
 ('EE278', 6.632184553752013)]
Largest positive coefficients:
[('PSYCH120', 8.171866137308884),
 ('EE278', 6.632184553752013),
 ('LINGUIST1', 4.104932555201978)]
Largest negative coefficients:
[('PSYCH90', -6.817632948191141),
 ('PSYCH164', -6.567715933999283),
 ('MS&E262', -6.4090183722776)]

***** C = 3.000000 *****
training accuracy: 0.840057636887608
test accuracy: 0.7702702702702703
Number of nonzero/zero coefficients: 159/479
Largest absolute coefficients:
[('CHEM135', -3.520306198558276),
 ('PSYCH90', -3.4159474957503884),
 ('LINGUIST1', 2.9365891508975226)]
Largest positive coefficients:
[('LINGUIST1', 2.9365891508975226),
 ('PSYCH120', 2.359489134782391),
 ('CS274', 2.190540914203022)]
Largest negative coefficients:
[('CHEM135', -3.520306